<a href="https://colab.research.google.com/github/Hoangthang017/CS331-338-431_nine-dash-line-detection/blob/continues_learn_about_Faster-RCNN/FeatureExtractionUsingAlexnet/Feature_Extraction_using_Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gg drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import library
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from imutils import paths
import os

import pandas as pd
import numpy as np

In [3]:
# define Alexnet Architecture
class Alexnet:
  def __init__(self, cs = 1000,  bp = "FC3"):
    self.classifer = cs
    self.breakPoint = bp
    self.model = None

  # setting model
  def define_model(self):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))

    # feature extraction in FC1
    if (self.breakPoint == "FC1"):
      self.model = model
      return

    model.add(Dropout(0.5))
    model.add(Dense(4096,activation='relu'))

    # feature extraction in FC2
    if (self.breakPoint == "FC2"):
      self.model = model
      return
    
    model.add(Dropout(0.5))
    model.add(Dense(self.classifer,activation='softmax'))
    self.model = model

  # predict 
  def predict(self, data):
    return self.model.predict(data)

  # get summary of model
  def summary(self):
    print(self.model.summary())

  # get model
  def get_model(self):
    return self.model

  # get/set break point
  def get_breakPoint(self):
    return self.breakPoint
  def set_breakPoint(self, bp):
    self.breakPoint = bp

  # get/set class
  def get_class(self):
    return self.classifer
  def set_class(self, c):
    self.classifer = c

In [4]:
# get feature from model
def getFeature(imagePath, model):
  # preprocess image
  image = load_img(imagePath,target_size=(227,227))
  img_array = img_to_array(image)
  img_array = np.expand_dims(img_array, axis=0)

  # get feature 
  feature = model.predict(img_array)
  feature = feature.reshape(-1)

  return feature

In [5]:
# feature extraction
def featureExtraction(imageDir, model):
  imagePaths = list(paths.list_images(imageDir))

  features = []
  labels = []

  for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    features.append(getFeature(imagePath,model))

    if (label == "Lines"):
      labels.append(1)
    else:
      labels.append(0)
  
  # create dataFrame
  # dataset = pd.DataFrame(
  # {
  #   "Featrues": features,
  #   "labels" : labels   
  # })
  return np.array(features),np.array(labels)

In [6]:
# main function
if __name__ ==  "__main__":
  pathDir = '/content/drive/MyDrive/Data Nine-Dash-Lines'

  # setting model
  model = Alexnet(cs=2,bp="FC1")
  model.define_model()
  model.summary()

  # feature extraction
  # FC1
  features_1 , labels_1 = featureExtraction(pathDir, model)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC1_features.txt",features_1)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC1_labels.txt",labels_1)

  # FC2
  model.set_breakPoint(bp="FC2")
  model.define_model()
  model.summary()
  features_2, labels_2 = featureExtraction(pathDir, model)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC2_features.txt",features_2)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC2_labels.txt",labels_2)

  # # show 
  # print(dt_feature)
  # print(dataset.shape)

  # print(dataset2)
  # print(dataset.shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8